Fluff to get the relative imports working in a notebook

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path

'C:\\Users\\micro_zo50ceu\\OneDrive - University College London\\BUCLSE'

# Timer

In [2]:
from UCLSE.custom_timer import CustomTimer

All of the objects we are about to explore need to agree about the current time. We pass them a simple timer. In the first instance this stops the need of passing a time variable around between objects. 

This is how we instantiate it:

In [3]:
timer=CustomTimer(start=0,end=10,step=1)

In [4]:
timer.duration

10.0

In [5]:
timer.get_time, timer.get_time_left

(0, 10.0)

In [6]:
print(timer)

time: 0 time left: 10.0 start: 0 end: 10 step: 1


To increment the time, we can call the following (it will only work if there is time remaining)

In [7]:
timer.next_period()
timer

time: 1 time left: 9.0 start: 0 end: 10 step: 1

# Exchange

In [8]:
from UCLSE.exchange import Exchange

An exchange object can be instantiated as follows:

In [9]:
exchange=Exchange(timer=timer)

In [10]:
print(exchange)

No orders in exchange order book


In [11]:
exchange.__dict__

{'bids': <UCLSE.exchange.Orderbook_half at 0x25e25615588>,
 'asks': <UCLSE.exchange.Orderbook_half at 0x25e256155c8>,
 'tape': [],
 'quote_id': 0,
 'timer': time: 1 time left: 9.0 start: 0 end: 10 step: 1,
 'name': 'exchange1',
 'record': False,
 'lob_call': deque([(0, 0)]),
 '_tape_index': 0}

The exchange is composed of two 'Orderbook_halfs'; each corresponding to ordered lists of bids and asks. 

Tape refers to the trades that have gone through the exchange to date. 

Quote_id is an internal counter which assigns a unique quote_id to each order placed on exchange

Timer is an object that when queried, tells the exchange what time it is. More on that later.

Lob call records when the last anonymised lob was constructed and gives it an ascending version number (to track different versions within the same time period)

# Trader

In [12]:
from UCLSE.traders import Trader

A trader object can be instantiated as follows:

In [13]:
henry=Trader(tid='Henry',exchange=exchange,timer=timer)

The particulars of the trader can be viewed by:

In [14]:
print(henry)

[TID: Henry type: None balance: 0 blotter: Empty DataFrame
Columns: []
Index: [] orders: OrderedDict() n_trades: 0 profitpertime: 0]


Or for a more in depth look:

In [15]:
henry.__dict__

{'ttype': None, 'tid': 'Henry', 'balance': 0, 'blotter': Empty DataFrame
 Columns: []
 Index: [], 'orders': [], 'orders_dic': OrderedDict(), 'history': False, 'orders_lookup': {}, 'n_orders': 0, 'n_quotes': 0, 'n_quote_limit': 1, 'profitpertime': 0, 'n_trades': 0, 'lastquote': {}, 'latency': 1, 'total_quotes': 0, 'timer': time: 1 time left: 9.0 start: 0 end: 10 step: 1, 'birthtime': 1, 'exchange': No orders in exchange order book, 'last_quote_time': 1}

The trader object is the parent class for multiple objects which represent specific types of trader. Included are Giveaway, ZIC (After Gode & Sunder 1993), Shaver, Sniper and ZIP (After Cliff 1997). 

Trader objects are differentiated by their getorder and respond methods. The former will produce an order when called, the second will update internal variables when called. 

If traders want to trade then they need to send orders to an exchange....

# orders

In [16]:
from UCLSE.exchange import Order

Traders submit orders to an exchange. An order is a namedtuple object which contains trader id, order type (Bid or Ask), price, quantity, time, quote_id and order_id.

In [17]:
order=Order('henry',otype='Bid',qty=1,price=100,time=1,oid=1)

Named tuples are cool:

In [18]:
print(order)

Order(tid='henry', otype='Bid', price=100, qty=1, time=1, qid=None, oid=1)


In [19]:
order.tid

'henry'

And they are mostly immutable, like normal tuples:

In [20]:
order.tid='John'

AttributeError: can't set attribute

Let's add the order to henry (from the perspective of a client giving him the order to execute)

In [21]:
henry.add_order(order)

('Proceed', None)

In [22]:
henry.__dict__

{'ttype': None, 'tid': 'Henry', 'balance': 0, 'blotter': Empty DataFrame
 Columns: []
 Index: [], 'orders': [], 'orders_dic': OrderedDict([(1,
               {'Original': Order(tid='henry', otype='Bid', price=100, qty=1, time=1, qid=None, oid=1),
                'submitted_quotes': [],
                'qty_remain': 1})]), 'history': False, 'orders_lookup': {}, 'n_orders': 1, 'n_quotes': 0, 'n_quote_limit': 1, 'profitpertime': 0, 'n_trades': 0, 'lastquote': {}, 'latency': 1, 'total_quotes': 0, 'timer': time: 1 time left: 9.0 start: 0 end: 10 step: 1, 'birthtime': 1, 'exchange': No orders in exchange order book, 'last_quote_time': 1}

The internal variables indicate that he has one order, and that it hasn't been submitted to the exchange (n_quotes=0)

Let's send an order to the exchange.

In [23]:
qid,trade_report,ammended_orders=henry.exchange.process_order(order)
print(qid,trade_report,ammended_orders)

0 None None


Note that the response to this is a number - the quote id for the new order and 'Addition' confirming the trade has been accepted.

The other variables are the trade report if there were any and any ammended orders (partial execution of orders)

You can see this record at the exchange either by the lob variable which is a dictionary of Order queues (called orderlist) indexed by price. 

The orderlist is a modified deque which holds the orders.

Deque - a datatype that efficiently allows us to append to the back and subtract from the front.

In [24]:
exchange.bids.lob

{100: OrderList([Order(tid='henry', otype='Bid', price=100, qty=1, time=1, qid=0, oid=1)])}

or in the dictionary of all orders, indexed by qid:

In [25]:
exchange.bids.q_orders

{0: Order(tid='henry', otype='Bid', price=100, qty=1, time=1, qid=0, oid=1)}

Finally we need to record with the trader that this quote has been submitted to exchange:

In [26]:
henry.add_order_exchange(order,qid)

This is because traders have 'Original' orders which do not change and they have submitted quotes at the exchange which they will probably alter over time.

In [27]:
henry.orders_dic

OrderedDict([(1,
              {'Original': Order(tid='henry', otype='Bid', price=100, qty=1, time=1, qid=None, oid=1),
               'submitted_quotes': [Order(tid='henry', otype='Bid', price=100, qty=1, time=1, qid=0, oid=1)],
               'qty_remain': 1})])

Let's illustrate how a trade gets executed. 

In [28]:
from UCLSE.test.utils import (yamlLoad, order_from_dic,build_df_from_dic_dic,build_lob_from_df,
                               lob_to_dic)
import pandas as pd
import os

First from a fixtures file, we can create a dataframe containing order information

In [56]:
cwd=os.getcwd()
fixture_name=os.path.join(os.path.dirname(cwd),'UCLSE','test','fixtures','exchange_fix.yml')
fixture_list=yamlLoad(fixture_name)
fixture_dic=fixture_list[0]

order_df=build_df_from_dic_dic(fixture_dic['input'])
order_df.set_index('index')

,otype,price,qid,qty,tid,time,oid
index,,,,,,,
0,Ask,107,1000,1,0,0,0
0,Bid,100,1000,1,0,0,1
1,Ask,108,1001,1,1,1,2
1,Bid,101,1001,1,1,1,3
2,Ask,109,1002,1,2,2,4
2,Bid,102,1002,1,2,2,5
3,Ask,110,1003,1,3,3,6
3,Bid,103,1003,1,3,3,7
4,Ask,111,1004,1,4,4,8


This function is in the utilities file but recreated here, it iterates through rows of DF, converts them to an order object then adds to exchange.

In [57]:
def build_lob_from_df(order_df,exch=None,necessary_cols=['tid','otype','price','qty','qid','oid']):
    ##adds orders from a df of orders, if supplied an exchange, will append them
    #else will create blank exchange
    #returns an exchange
    order_df=order_df[necessary_cols]

    if exch is None:
        exch=Exchange(timer=CustomTimer())

    order_list=[]
    for index, row in order_df.iterrows():
        o=row.to_dict()
        #o.pop('index')
        exch.process_order(Order(time=exch.time,**o),verbose=False)

    return exch

In [58]:
exchange=build_lob_from_df(order_df)

And we can see that what the LOB looks like:

In [59]:
print(exchange)

                        tid     
otype                   Ask  Bid
price time qid oid qty          
100   0    1   1   1    NaN  0.0
101   0    3   3   1    NaN  1.0
102   0    5   5   1    NaN  2.0
103   0    7   7   1    NaN  3.0
104   0    9   9   1    NaN  4.0
105   0    11  11  1    NaN  5.0
107   0    0   0   1    0.0  NaN
108   0    2   2   1    1.0  NaN
109   0    4   4   1    2.0  NaN
110   0    6   6   1    3.0  NaN
111   0    8   8   1    4.0  NaN
112   0    10  10  1    5.0  NaN


Next period the trader receives the following order:

In [60]:
timer.next_period()
henry=Trader(tid='Henry',exchange=exchange,timer=timer,time=0,history=True) #time input manually specifies trader birthtime
new_order=Order(tid='Henry', otype='Bid', price=109, qty=5, qid=None,time=henry.time, oid=50)
new_order

Order(tid='Henry', otype='Bid', price=109, qty=5, time=4, qid=None, oid=50)

Add to internal records, submit to exchange, receive qid - record quote internally, receive information about any executions

In [61]:
henry.add_order(new_order) #add to trader records
qid,trade_report,ammended_orders=henry.exchange.process_order(new_order,verbose=True) #send to exchange
henry.add_order_exchange(new_order,qid) #confirm order placement with trader

QUID: order.quid=12
RESPONSE: Addition
Bid  leg 0  lifts best  Ask 109
counterparty 0 price 107
order partially filled, new ammended one  0 12.000001 Order(tid='Henry', otype='Bid', price=109, qty=4, time=4, qid=12.000001, oid=50)
Bid  leg 1  lifts best  Ask 109
counterparty 1 price 108
order partially filled, new ammended one  1 12.000002 Order(tid='Henry', otype='Bid', price=109, qty=3, time=4, qid=12.000002, oid=50)
Bid  leg 2  lifts best  Ask 109
counterparty 2 price 109
order partially filled, new ammended one  2 12.000003 Order(tid='Henry', otype='Bid', price=109, qty=2, time=4, qid=12.000003, oid=50)


In [62]:
henry.orders_dic

OrderedDict([(50,
              {'Original': Order(tid='Henry', otype='Bid', price=109, qty=5, time=4, qid=None, oid=50),
               'submitted_quotes': [Order(tid='Henry', otype='Bid', price=109, qty=5, time=4, qid=12, oid=50)],
               'qty_remain': 5})])

The trade report details any executions

In [63]:
trade_report

[{'type': 'Trade',
  'tape_time': 0,
  'tidx': 0,
  'price': 107,
  'party1': 0,
  'party2': 'Henry',
  'qty': 1,
  'p1_qid': 0,
  'p2_qid': 12.0},
 {'type': 'Trade',
  'tape_time': 0,
  'tidx': 1,
  'price': 108,
  'party1': 1,
  'party2': 'Henry',
  'qty': 1,
  'p1_qid': 2,
  'p2_qid': 12.000001},
 {'type': 'Trade',
  'tape_time': 0,
  'tidx': 2,
  'price': 109,
  'party1': 2,
  'party2': 'Henry',
  'qty': 1,
  'p1_qid': 4,
  'p2_qid': 12.000002}]

Because there were 3 legs to the execution, there are 3 ammendments:

In [64]:
ammended_orders

[AmmendedOrderRecord(tid='Henry', qid=12.000001, order=Order(tid='Henry', otype='Bid', price=109, qty=4, time=4, qid=12.000001, oid=50)),
 AmmendedOrderRecord(tid='Henry', qid=12.000002, order=Order(tid='Henry', otype='Bid', price=109, qty=3, time=4, qid=12.000002, oid=50)),
 AmmendedOrderRecord(tid='Henry', qid=12.000003, order=Order(tid='Henry', otype='Bid', price=109, qty=2, time=4, qid=12.000003, oid=50))]

The fills and the ammendments need to be reported to the trader.

In [65]:
def bookkeeping(trader,trade_report,ammended_orders):
    
    for trade,ammended_order in zip(trade_report,ammended_orders):
        active=False
        if  trade['party2']==trader.tid: active=True
            

        trader.bookkeep(trade,new_order,True,time=trader.time,active=active)

        ammend_tid=ammended_order.tid
        if ammend_tid==trader.tid:
            ammend_qid=ammended_order.qid
            trader.add_order_exchange(ammended_order.order,ammend_qid)
            
bookkeeping(henry,trade_report,ammended_orders)

Order(tid='Henry', otype='Bid', price=109, qty=5, time=4, qid=None, oid=50) profit=2 balance=2 profit/time=0
Order(tid='Henry', otype='Bid', price=109, qty=5, time=4, qid=None, oid=50) profit=1 balance=3 profit/time=0
Order(tid='Henry', otype='Bid', price=109, qty=5, time=4, qid=None, oid=50) profit=0 balance=3 profit/time=0


Running profit is stored in the balance attribute and the executions are stored in the blotter, it is the difference between the original order price and the executed price.

In [66]:
assert henry.balance==pd.DataFrame(henry.blotter).profit.sum()
henry.balance

3

In [67]:
henry.blotter

  type  tape_time  tidx  price  party1 party2  qty  p1_qid     p2_qid  oid    tid  order qty  order_issue_time  profit   BS   status
0  Bid          0     0    107       0  Henry    1       0  12.000000   50  Henry          5                 4       2  Buy  partial
1  Bid          0     1    108       1  Henry    1       2  12.000001   50  Henry          4                 4       1  Buy  partial
2  Bid          0     2    109       2  Henry    1       4  12.000002   50  Henry          3                 4       0  Buy  partial

The original order was for 5, 3 units have been executed so there should be a remaining order of qty 2. Note that qid is incremented for the ammended order on exchange.

In [68]:
henry.orders_dic

OrderedDict([(50,
              {'Original': Order(tid='Henry', otype='Bid', price=109, qty=5, time=4, qid=None, oid=50),
               'submitted_quotes': [Order(tid='Henry', otype='Bid', price=109, qty=5, time=4, qid=12, oid=50),
                Order(tid='Henry', otype='Bid', price=109, qty=4, time=4, qid=12.000001, oid=50),
                Order(tid='Henry', otype='Bid', price=109, qty=3, time=4, qid=12.000002, oid=50),
                Order(tid='Henry', otype='Bid', price=109, qty=2, time=4, qid=12.000003, oid=50)],
               'qty_remain': 2})])

Check that the exchange agrees:

In [69]:
print(exchange)

                              tid       
otype                         Ask    Bid
price time qid       oid qty            
100   0    1.000000  1   1    NaN      0
101   0    3.000000  3   1    NaN      1
102   0    5.000000  5   1    NaN      2
103   0    7.000000  7   1    NaN      3
104   0    9.000000  9   1    NaN      4
105   0    11.000000 11  1    NaN      5
109   4    12.000003 50  2    NaN  Henry
110   0    6.000000  6   1      3    NaN
111   0    8.000000  8   1      4    NaN
112   0    10.000000 10  1      5    NaN


The order book will act how you would expect so if another Buy order is entered at 109, it will get a lower priority:

In [70]:
john=Trader(tid='John',exchange=exchange,timer=timer,time=0)
new_order=Order(tid='John', otype='Bid', price=109, qty=1, qid=None,time=john.time, oid=51)
john.add_order(new_order) #add to trader records
qid,trade_report,ammended_orders=exchange.process_order(new_order,verbose=True) #send to exchange
john.add_order_exchange(new_order,qid) #confirm order placement with trader

QUID: order.quid=16
RESPONSE: Addition


In [71]:
print(exchange)

                              tid       
otype                         Ask    Bid
price time qid       oid qty            
100   0    1.000000  1   1    NaN      0
101   0    3.000000  3   1    NaN      1
102   0    5.000000  5   1    NaN      2
103   0    7.000000  7   1    NaN      3
104   0    9.000000  9   1    NaN      4
105   0    11.000000 11  1    NaN      5
109   4    12.000003 50  2    NaN  Henry
           16.000000 51  1    NaN   John
110   0    6.000000  6   1      3    NaN
111   0    8.000000  8   1      4    NaN
112   0    10.000000 10  1      5    NaN


Check with a sell order of quantity 2 at 109 that Henry is completed not John.

In [72]:
paul=Trader(tid='Paul',exchange=exchange,timer=timer,time=0)
new_order=Order(tid='Paul', otype='Ask', price=109, qty=2, qid=None,time=paul.time, oid=52)
paul.add_order(new_order) #add to trader records
qid,trade_report,ammended_orders=exchange.process_order(new_order,verbose=True) #send to exchange
paul.add_order_exchange(new_order,qid) #confirm order placement with trader

QUID: order.quid=17
RESPONSE: Addition
Ask  leg 0  lifts best  Bid 109
counterparty Henry price 109


In [73]:
print(exchange)

                        tid      
otype                   Ask   Bid
price time qid oid qty           
100   0    1   1   1    NaN     0
101   0    3   3   1    NaN     1
102   0    5   5   1    NaN     2
103   0    7   7   1    NaN     3
104   0    9   9   1    NaN     4
105   0    11  11  1    NaN     5
109   4    16  51  1    NaN  John
110   0    6   6   1      3   NaN
111   0    8   8   1      4   NaN
112   0    10  10  1      5   NaN


In [74]:
bookkeeping(henry,trade_report,ammended_orders)

Order(tid='Henry', otype='Bid', price=109, qty=5, time=4, qid=None, oid=50) profit=0 balance=3 profit/time=0


Since the trade is completed, the trader's orders_dic is wiped

In [48]:
henry.orders_dic

OrderedDict()

But the history of the order is moved

In [49]:
henry.orders_dic_hist

AttributeError: 'Trader' object has no attribute 'orders_dic_hist'

and executions are stored on the blotter

In [50]:
henry.blotter

  type  tape_time  tidx  price party1 party2  qty     p1_qid     p2_qid  oid    tid  order qty  order_issue_time  profit   BS   status
0  Bid          0     0    107      0  Henry    1   0.000000  12.000000   50  Henry          5                 2       2  Buy  partial
1  Bid          0     1    108      1  Henry    1   2.000000  12.000001   50  Henry          4                 2       1  Buy  partial
2  Bid          0     2    109      2  Henry    1   4.000000  12.000002   50  Henry          3                 2       0  Buy  partial
3  Bid          0     3    109  Henry   Paul    2  12.000003  17.000000   50  Henry          2                 2       0  Buy     full